In [1]:
# Draw 2D bounding box on image 
import cv2
import time
import os
from util_kitti import init_img_plt, draw_2Dbox, kitti_label_file_parser, draw_corner_2D, draw_birdeyes, kitti_calib_file_parser
import random 
import copy
random.seed(2222)
NUM_IMAGE_SHOW = 20

# TODO 
# images files 
IMG_DIR = "/home/lab530/KenYu/kitti/training/image_2/"
# Anotations files 
ANO_DIR = "/home/lab530/KenYu/kitti/training/label_2/"
# Prediction files
PRD_DIR = "/home/lab530/KenYu/pytorch-retinanet/checkpoint/2D_detection_crop_feature_map_pretrain_result/" # "/home/lab530/KenYu/pytorch-retinanet/checkpoint/2D_detection_3/result/"
# 
CAR_DIR = "/home/lab530/KenYu/kitti/training/calib/"

/home/lab530/anaconda3/envs/KenYuGAC/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_names = [i.split('.')[0] for i in os.listdir(PRD_DIR)]

random.shuffle(img_names)
chose_name = img_names[:NUM_IMAGE_SHOW]
# chose_img  = [cv2.imread(os.path.join(IMG_DIR, f"{i}.png")) for i in chose_name]

# Transform image 
chose_img = []
chose_P2  = []
for name in chose_name:
    img = cv2.imread(os.path.join(IMG_DIR, f"{name}.png"))
    img_ori_h, img_ori_w, _ = img.shape
    
    P2 = kitti_calib_file_parser(os.path.join(CAR_DIR, f"{name}.txt"))

    # # Resize
    # P2[0, :] *= 1280  / img_ori_w
    # P2[1, :] *= 384   / img_ori_h
    
    chose_P2.append(P2)
    
    # img = cv2.resize(img, (1280, 384))
    
    chose_img.append(img)

axs = init_img_plt(chose_img, titles = chose_name)

for i, img_name in enumerate(chose_name):
    
    # Load predictions
    with open(os.path.join(PRD_DIR, f"{img_name}.txt")) as f:
        lines = f.read().splitlines()
        lines = list(lines for lines in lines if lines) # Delete empty lines
    
    # Draw predictions on image
    for l in lines:
        l_list = l.split()
        if float(l_list[-1]) > 0.5: 
            draw_2Dbox(axs[i][0], (float(l_list[4]), float(l_list[5]), float(l_list[6]), float(l_list[7])), color = (1,0,0))
    
    # Load annotation
    # with open(os.path.join(ANO_DIR, f"{img_name}.txt")) as f:
    #     lines = f.read().splitlines()
    #     lines = list(lines for lines in lines if lines) # Delete empty lines
    objs = kitti_label_file_parser(os.path.join(ANO_DIR, f"{img_name}.txt"), tf_matrix = chose_P2[i])

    # Draw annotation on image
    for obj in objs:
        if obj.category == "Car":
            draw_2Dbox(axs[i][0], (obj.xmin, obj.ymin, obj.xmax, obj.ymax), color = (0,1,0))
            draw_birdeyes(axs[i][1], obj, color = (0,1,0), title = "123", is_print_confident = False)
            # draw_corner_2D(axs[i][0], obj.corner_2D, color = (0,1,0), is_draw_front = True)



ValueError: Number of rows must be a positive integer, not 0

<Figure size 1800x0 with 0 Axes>